## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- (Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt)
- (Extraction de termes MeSH et SNOMED présents dans les données)

### **Lire le corpus** 

In [244]:
lng = 'fr'
acteur = 'msss'
tag = 'sante-mentale'
file = acteur + '/' + acteur + '_' + tag + '.csv'

In [245]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [246]:
def lire_corpus(csv_file, langue=lng):
    base_path = '../03-corpus/2-data/'
    file_path = path.join(base_path, '1-' + langue, csv_file)

    #encoding='ISO 8859-1'
    with open(file_path, "r", encoding='utf-8') as f:
        data = read_csv(file_path, encoding='utf-8', sep=',')
        data = data[~data["url"].str.contains('pdf')] # Problèmes 
        data = data['text']
        
        
    return data

In [247]:
data = lire_corpus(file) 

### **Nettoyage**

In [248]:
punct = '[!#$%&\(\)•►*+,-\/:;<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '[a-zA-Z]+\d+[a-zA-Z]'
phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data]
text = [re.sub(spaces, ' ', t).replace("  ", " " ) for t in text]
text = [re.sub(phones, ' ', t).replace("  ", " " ) for t in text]
text = [re.sub(postals, ' ', t).replace("  ", " " ) for t in text]
text = [t.replace("  ", " ") for t in text]
text = [re.sub(punct, ' ', t).replace("  ", " " ) for t in text]

In [249]:
nb_docs = len(text)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 129 documents.


### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [250]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 129 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [251]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' MWE_STOP ').replace('  ', " ")
    return corpus

In [252]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [253]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour "forcer" le Tree Tagger à tokeniser comme on veut
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))
    return tokens

tokens = tok(corpus)
corpus = " ".join(tokens).replace("' ", "'")

In [255]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append((t.split('\t')[0], t.split('\t')[1]))
        except Exception as e:
            output.append(('MWE_STOP', 'NAM'))

    return output


In [256]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]
tagged[:25]

[('gestion', 'NOM'),
 ('clinique', 'ADJ'),
 ('de', 'PRP'),
 ("l'", 'DET:ART'),
 ('épisode', 'NOM'),
 ('de', 'PRP'),
 ('soins', 'NOM'),
 ('santé', 'NOM'),
 ('mentale', 'ADJ'),
 ('professionnels', 'ADJ'),
 ('de', 'PRP'),
 ('la', 'DET:ART'),
 ('santé', 'NOM'),
 ('msss', 'ADJ'),
 ('MWE_STOP', 'NOM'),
 ('ministère', 'NOM'),
 ('de', 'PRP'),
 ('la', 'DET:ART'),
 ('santé', 'NOM'),
 ('et', 'KON'),
 ('des', 'PRP:det'),
 ('services', 'NOM'),
 ('sociaux', 'ADJ'),
 ('ministère', 'NOM'),
 ('MWE_STOP', 'NAM')]

In [257]:
# Si on veut lemmatiser
### **Mapping POS Tags** (FRMG)

#Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

#file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

#with open(file_path) as f:
#    csv = read_csv(f)

#treeTag = [term for term in csv['TreeTagger'].tolist()] 
#lefff = [term for term in csv['Lefff'].tolist()]

#mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [258]:
# tagged = [[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(corpus)]

In [259]:
# Si on veut lemmatiser
### **Lemmatisation** (FrenchLefffLemmatizer)
#https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

#from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
#lemmatizer = FrenchLefffLemmatizer()

#lemmas = []
#for term in tagged:
#    term_l = []
#    if lemmatizer.lemmatize(term[0], term[1]) == []:
#        term_l = (lemmatizer.lemmatize(term[0]), term[1])
    
    # elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
    #     term_l  = (lemmatizer.lemmatize(term[0], term[1]), term[1])

    # else:
    #     term_l = tuple(lemmatizer.lemmatize(term[0], term[1])[0])
    
    # lemmas.append(term_l)

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [260]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=2, max_len=6))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [261]:
ngrammes = extr_ngrams(tagged)

Avant filtrage, on a 688150 ngrammes.


In [262]:
ngrammes

[(('gestion', 'NOM'), ('clinique', 'ADJ')),
 (('gestion', 'NOM'), ('clinique', 'ADJ'), ('de', 'PRP')),
 (('gestion', 'NOM'), ('clinique', 'ADJ'), ('de', 'PRP'), ("l'", 'DET:ART')),
 (('gestion', 'NOM'),
  ('clinique', 'ADJ'),
  ('de', 'PRP'),
  ("l'", 'DET:ART'),
  ('épisode', 'NOM')),
 (('gestion', 'NOM'),
  ('clinique', 'ADJ'),
  ('de', 'PRP'),
  ("l'", 'DET:ART'),
  ('épisode', 'NOM'),
  ('de', 'PRP')),
 (('clinique', 'ADJ'), ('de', 'PRP')),
 (('clinique', 'ADJ'), ('de', 'PRP'), ("l'", 'DET:ART')),
 (('clinique', 'ADJ'), ('de', 'PRP'), ("l'", 'DET:ART'), ('épisode', 'NOM')),
 (('clinique', 'ADJ'),
  ('de', 'PRP'),
  ("l'", 'DET:ART'),
  ('épisode', 'NOM'),
  ('de', 'PRP')),
 (('clinique', 'ADJ'),
  ('de', 'PRP'),
  ("l'", 'DET:ART'),
  ('épisode', 'NOM'),
  ('de', 'PRP'),
  ('soins', 'NOM')),
 (('de', 'PRP'), ("l'", 'DET:ART')),
 (('de', 'PRP'), ("l'", 'DET:ART'), ('épisode', 'NOM')),
 (('de', 'PRP'), ("l'", 'DET:ART'), ('épisode', 'NOM'), ('de', 'PRP')),
 (('de', 'PRP'),
  ("l'", '

### **Extraction des patrons syntaxiques**

*LONG*

In [263]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = [t[0] for t in ng]
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [264]:
phrases = extract_patterns(ngrammes)
# phrases_lemmatized = extract_patterns(ngrammes_lemmatized)

In [265]:
def freq(phrases):
    return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

In [266]:
frequencies = freq(phrases)

In [267]:
frequencies

FreqDist({'de la': 1985, "de l'": 1512, 'santé mentale': 1397, 'santé et': 967, 'la santé': 949, 'de la santé': 731, 'des services': 673, "l'urgence": 646, "à l'": 627, 'services sociaux': 613, ...})

In [268]:
phrases[:5]

[[['gestion', 'clinique'], ['NOM', 'ADJ']],
 [['gestion', 'clinique', 'de'], ['NOM', 'ADJ', 'PRP']],
 [['gestion', 'clinique', 'de', "l'"], ['NOM', 'ADJ', 'PRP', 'DET:ART']],
 [['gestion', 'clinique', 'de', "l'", 'épisode'],
  ['NOM', 'ADJ', 'PRP', 'DET:ART', 'NOM']],
 [['gestion', 'clinique', 'de', "l'", 'épisode', 'de'],
  ['NOM', 'ADJ', 'PRP', 'DET:ART', 'NOM', 'PRP']]]

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [269]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [270]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    return [term for term in x if not 'MWE_STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords and not 'NUM' in term[1][:-4] and term[1][0] == 'NOM']

In [271]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [272]:
def filter_len(x):
    return [term for term in x if \
        len(term[0][0]) > 2 and len(term[0][0]) < 18 and \
        len(term[0][-1]) > 2 and len(term[0][-1]) < 18]

In [273]:
phrases = filter_len(phrases)

In [274]:
def filter_freq(x):
    return [term for term in x if frequencies[" ".join(term[0]).replace("' ", "'")] > 5]

In [275]:
phrases = filter_freq(phrases)

In [276]:
phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [277]:
for phrase in phrases:
    phrase.append(frequencies[phrase[0]])

In [278]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 61022 occurrences de ngrammes.


In [279]:
phrases

[['gestion clinique', 'NOM ADJ', 66],
 ["gestion clinique de l'épisode", 'NOM ADJ PRP DET:ART NOM', 64],
 ['épisode de soins', 'NOM PRP NOM', 168],
 ['santé mentale', 'NOM ADJ', 1397],
 ['santé mentale professionnels', 'NOM ADJ ADJ', 43],
 ['santé mentale professionnels de la santé',
  'NOM ADJ ADJ PRP DET:ART NOM',
  43],
 ['santé msss', 'NOM ADJ', 82],
 ['ministère de la santé', 'NOM PRP DET:ART NOM', 271],
 ['santé et des services', 'NOM KON PRP:det NOM', 321],
 ['santé et des services sociaux', 'NOM KON PRP:det NOM ADJ', 321],
 ['santé et des services sociaux ministère',
  'NOM KON PRP:det NOM ADJ NOM',
  84],
 ['services sociaux', 'NOM ADJ', 613],
 ['services sociaux ministère', 'NOM ADJ NOM', 84],
 ['covid 19 répertoires', 'NOM NUM NOM', 166],
 ['ministère de la santé', 'NOM PRP DET:ART NOM', 271],
 ['santé et des services', 'NOM KON PRP:det NOM', 321],
 ['santé et des services sociaux', 'NOM KON PRP:det NOM ADJ', 321],
 ['services sociaux', 'NOM ADJ', 613],
 ['services sociaux i

In [280]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

,Expression,Patron syntaxique,Fréquence
0,gestion clinique,NOM ADJ,66
1,gestion clinique de l'épisode,NOM ADJ PRP DET:ART NOM,64
2,épisode de soins,NOM PRP NOM,168
3,santé mentale,NOM ADJ,1397
4,santé mentale professionnels,NOM ADJ ADJ,43
...,...,...,...
61017,habitudes d'utilisation répondre,NOM PRP NOM VER:infi,82
61018,habitudes d'utilisation répondre répondre,NOM PRP NOM VER:infi VER:infi,82
61019,utilisation répondre,NOM VER:infi,82
61020,utilisation répondre répondre,NOM VER:infi VER:infi,82


In [281]:
def tabCSV(phrases):
    base_path = '../04-filtrage/output/'
    tab = DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"]).drop_duplicates()
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)


    # file_path = path.join(base_path, tag, tag)    
    # Path(file_path).mkdir(parents=True, exist_ok=True)

    #tab.to_csv(file_path + '_ngrams.csv') - On va seulement garder les collocations significatives comme output

    return tab.values.tolist()

In [282]:
phrases = tabCSV(phrases)
# phrases_lemmatized = tabCSV(phrases_lemmatized, '_n-grams-lemmatized.csv')

In [283]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(phrases)))

Après filtrage, on a 2758 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [284]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:200] # On prend les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [285]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NOM NOM' in t[1] and not 'NUM' in patterns[1]] # 
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [286]:
terms = filter_patterns(phrases)
terms[:50]

[['santé mentale', 'NOM ADJ', 1397],
 ['services sociaux', 'NOM ADJ', 613],
 ['ministère de la santé', 'NOM PRP DET:ART NOM', 271],
 ['services de santé', 'NOM PRP NOM', 199],
 ['professionnels de la santé', 'NOM PRP DET:ART NOM', 170],
 ['épisode de soins', 'NOM PRP NOM', 168],
 ['covid 19 répertoires', 'NOM NUM NOM', 166],
 ['indicateurs de gestion', 'NOM PRP NOM', 158],
 ['répertoire des indicateurs', 'NOM PRP:det NOM', 148],
 ['répertoire des indicateurs de gestion', 'NOM PRP:det NOM PRP NOM', 148],
 ['partie des installations de la région',
  'NOM PRP:det NOM PRP DET:ART NOM',
  128],
 ['partie des installations', 'NOM PRP:det NOM', 128],
 ['installations de la région', 'NOM PRP DET:ART NOM', 128],
 ['maladies chroniques', 'NOM ADJ', 112],
 ['santé et services sociaux', 'NOM KON NOM ADJ', 108],
 ['santé et services', 'NOM KON NOM', 108],
 ['gestion en santé', 'NOM PRP NOM', 102],
 ['indicateurs de gestion en santé', 'NOM PRP NOM PRP NOM', 102],
 ['promotion de la santé', 'NOM PRP 

In [287]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 55 % des termes
On avait 2758 ngrammes, on en a maintenant 1248.


In [288]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [289]:
terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [290]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [291]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 1248 ngrammes.


In [292]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [293]:
fd_tokens

FreqDist({'de': 10039, 'et': 5093, 'la': 4031, 'des': 3650, "l'": 3326, 'santé': 3004, 'MWE_STOP': 2348, "d'": 2325, 'en': 2311, 'les': 2231, ...})

In [294]:
def llr_ngrammes(n):
    llr = []

    for i in range(2, n+1):
        ngrammes = set([tuple(tokenizer_re.tokenize(term[0])) for term in terms if len(tokenizer_re.tokenize(term[0])) == i])
        fd = nltk.FreqDist(ngrams(tokens, n=i))
        fd_prior = nltk.FreqDist(ngrams(tokens, n=i-1))
        
        for t in ngrammes:
            c_prior = fd_prior[t[:i-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[i-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = fd[t]             # Le ngramme lui-même P(w1w2w3..wn)

            res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
            p = chi2.sf(res, 1) # 1 degrees of freedom
            #if res == float('-inf') :
            #    res = 50000

            if p < 0.001 or (res == float('-inf')):
                llr.append({'Collocation' : " ".join(t).replace("' ", "'"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p})

    return llr


In [295]:
terms = llr_ngrammes(6)

In [298]:
terms
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms]

[{'Collocation': 'informations générales',
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 26,
  'LLR': 370.03237614706444,
  'p-value': 1.841385910137856e-82},
 {'Collocation': 'efforts importants',
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 6,
  'LLR': 110.02929742070202,
  'p-value': 9.65531296349499e-26},
 {'Collocation': 'suivi intensif',
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 42,
  'LLR': 599.5798212460187,
  'p-value': 2.0662901608700536e-132},
 {'Collocation': 'maladies chroniques',
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 112,
  'LLR': 1427.6291150624236,
  'p-value': 0.0},
 {'Collocation': 'gestion autonome',
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 12,
  'LLR': 142.776637819991,
  'p-value': 6.57761186872895e-33},
 {'Collocation': 'dépendance ciusss',
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 11,
  'LLR': 89.77622010101322,
  'p-value': 2.6668047922538508e-21},
 {'Collocation': 'perspective ministérielle',
  'Structur

In [299]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len_prior - len(terms)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len_prior - len(terms)) / len_prior *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 22 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 2 % de nos n-grammes.


In [300]:
DataFrame(terms).sort_values(by = "Fréquence", ascending=False)

,Collocation,Structure syntaxique,Fréquence,LLR,p-value
147,santé mentale,NOM ADJ,1397,-inf,1.000000e+00
142,services sociaux,NOM ADJ,613,-inf,1.000000e+00
835,ministère de la santé,NOM PRP DET:ART NOM,271,-inf,1.000000e+00
319,services de santé,NOM PRP NOM,199,1047.001635,1.091531e-229
863,professionnels de la santé,NOM PRP DET:ART NOM,170,1288.266157,4.010136e-282
...,...,...,...,...,...
387,application des mesures,NOM PRP:det NOM,6,64.565665,9.336825e-16
953,facettes d'une intégration,NOM PRP DET:ART NOM,6,86.848289,1.171733e-20
952,collaboration en troubles concomitants,NOM PRP NOM ADJ,6,114.541949,9.914649e-27
951,services dans la communauté,NOM PRP DET:ART NOM,6,91.535591,1.096030e-21


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; en tokenisant, on supprimer les frontières entre les mots qui sont des tirets ou autres caractères qu'un espace ou un apostrophe ; ça fait en sorte qu'on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

*LONG*  
Est-ce qu'il y a moyen de rendre ça plus efficace ?

In [301]:
dfs = {term['Collocation']: len([doc for doc in text if term['Collocation'] in doc]) for term in terms}

In [302]:
max_df = round(0.87 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 85% des documents 
max_df

112

In [303]:
test = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve

In [304]:
zeros = {term for term in dfs if dfs[term] == 0}
dfs_100 = {term for term in dfs if dfs[term] >= max_df}

In [305]:
dfs_100

{'ministère de la santé',
 'réseau professionnels',
 'santé mentale',
 'services de santé',
 'services sociaux'}

In [306]:
terms = [t for t in terms if dfs[t['Collocation']] < max_df ]

In [307]:
#colloc = [term['Collocation'] for term in terms]
#len(colloc)

In [308]:
#for term in terms:
 #   term['DF'] = dfs[term['Collocation']] 
DataFrame(terms).sort_values(by="Fréquence", ascending=False)

,Collocation,Structure syntaxique,Fréquence,LLR,p-value
858,professionnels de la santé,NOM PRP DET:ART NOM,170,1288.266157,4.010136e-282
431,épisode de soins,NOM PRP NOM,168,-inf,1.000000e+00
326,covid 19 répertoires,NOM NUM NOM,166,-inf,1.000000e+00
653,indicateurs de gestion,NOM PRP NOM,158,-inf,1.000000e+00
1019,répertoire des indicateurs de gestion,NOM PRP:det NOM PRP NOM,148,-inf,1.000000e+00
...,...,...,...,...,...
997,idées suicidaires principales sources,NOM ADJ ADJ NOM,6,88.399372,5.348779e-21
502,santé mentale 2014,NOM ADJ NUM,6,17.784547,2.473843e-05
496,description des ateliers,NOM PRP:det NOM,6,70.363013,4.933633e-17
493,quête de valorisation,NOM PRP NOM,6,123.489415,1.089660e-28


In [309]:
df = DataFrame(terms)
df.sort_values(['Fréquence'], 
            axis=0,
            ascending=[False], 
            inplace=True)

output_path = path.join('../04-filtrage/output/', tag + '_significant-collocations.csv') 
df.to_csv(output_path)

In [310]:
list_terms = [term['Collocation'] for term in terms]

In [311]:
terms = DataFrame(terms)

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [ ]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'") for ng in ngrammes]

In [ ]:
extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence'])
kwic = {w : [] for w in kw} 

In [ ]:
for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [ ]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [ ]:
for term in kw:
    df = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = pd.concat([extrant, df])


extrant = extrant[extrant['Fréquence'] > 30] 

file_path = '../04-filtrage/output/'
file_path = path.join(file_path, tag, tag)


#extrant.to_csv(file_path + '_KWIC' +'.csv')

NameError: name 'pd' is not defined

### **Extraction de termes MeSH**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
# extr_mesh = tokenizer_mesh.tokenize([t['Collocation'] for t in terms])

# **MODIF**

In [ ]:
# MODIFICATION À APPORTER : dans l'extrant où on voit les termes mesh, ajouter la fréquence

termes_mesh[:15]

for t in termes_mesh:
    print(t, corpus.count(t))

In [ ]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)

In [ ]:
file_path = '../04-filtrage/output/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)

df = DataFrame(termes_mesh)
df.to_csv(file_path + '_MeSH.csv')

### **Extraction de termes SNOMED**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/SNOMED_fr.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

In [ ]:
extr_sm = tokenizer_sm.tokenize([t[0] for t in terms])

In [ ]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

In [ ]:
file_path = '../04-filtrage/output' 
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag) 

else :
    file_path = path.join(file_path, acteur, acteur)


df = DataFrame(termes_sm)

df.to_csv(file_path + '_SNOMED.csv')